In [1]:
# Importing necessary library
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import os
from scipy import signal
from stingray import lightcurve
import sys
from stingray import Bispectrum
import warnings
import csv
warnings.filterwarnings('ignore')
%matplotlib inline

c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:25: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")
c:\Users\fulky\anaconda3\envs\skripsi\lib\site-packages\stingray\utils.py:40: UserWarning: Numba not installed. Faking it
  warnings.warn("Numba not installed. Faking it")


In [2]:
# file that will be skipped because of information loss
zeros_test = ['co2a0000368_91.csv', 'co2c0000341_26.csv']
zeros_train = ['co2a0000368_0.csv', 'co2a0000368_1.csv', 'co2a0000368_2.csv', 'co2a0000368_3.csv', 'co2a0000368_4.csv', 'co2a0000368_5.csv', 'co2c0000341_27.csv']

### Menghitung Matriks Cumulant orde ke-3

In [4]:
def calcCumulantOrde3(df_data, t, lag):
    # Compute the bispectrum of the signal
    f, t, Zxx = signal.stft(df_data, 256)

    # Plot the bispectrum using contour plots
    # plt.contour(bs.freq, bs.freq, bs.bispec_mag)
    # plt.xlabel('f1')
    # plt.ylabel('f2')
    # plt.show()

    # # Plot the bispectrum using mesh plots
    # fig = plt.figure()
    # ax = fig.add_subplot(111, projection='3d')
    # X, Y = np.meshgrid(bs.freq, bs.freq)
    # ax.plot_surface(X, Y, bs.bispec_mag)
    # ax.set_xlabel('f1')
    # ax.set_ylabel('f2')
    # ax.set_zlabel('Bispectrum')
    # plt.show()

    return Zxx

### Persiapan data

In [6]:
# Define sampling frequency
fs = 256
t = np.arange(0, 1, 1/fs)

def get_csv_EEG(filename):
    # Load data from CSV
    data = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=range(3,259))
    channel_name = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')
    
    df_data = pd.DataFrame(data.T, columns=channel_name)

    df_data = df_data.drop(columns=['X', 'Y', 'nd'])

    return df_data, df_data.columns


### Perhitungan Bispectrum

In [6]:
def extract_feature(directory, lag):
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(directory.replace('CSV', 'bispectrum')+"_" + str(lag),foldername).lower()
            files = os.listdir(folder)
            for filename in files:
                if filename in zeros_train or filename in zeros_test:
                    continue
                rel_path = os.path.join(directory, foldername, filename)
                if 'metadata' in filename.lower():
                    continue
                trial_number = filename.split('.')[0].split('_')[1]
                des_file = foldername+'_'+ str(trial_number) + '_bispectrum' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                df_data, channel_name = get_csv_EEG(rel_path)
                bispectrum = []
                for channel in channel_name:
                    magnitude = calcCumulantOrde3(df_data[channel], t, 256)
                    bispectrum.append(magnitude)
                bispectrum = np.array(bispectrum)
                np.save(des_path, bispectrum)
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
        


### Main Program

In [10]:
df_data, channel_name = get_csv_EEG('../SMNI_CMI_TEST_CSV\co2a0000364\co2a0000364_30.csv')
bispectrum = []
for channel in channel_name:
    magnitude = calcCumulantOrde3(df_data[channel], t, 256)
    bispectrum.append(magnitude)
bispectrum = np.array(bispectrum)
bispectrum[0]

array([[ 2.35268300e+00+0.00000000e+00j,  8.48440379e+00+0.00000000e+00j,
         2.79544446e+00+0.00000000e+00j],
       [-3.72809946e-01+1.87173277e+00j, -5.10519916e+00-1.85169222e+00j,
        -1.39611288e+00-1.89734737e+00j],
       [-7.61795513e-01-5.25312285e-01j,  1.09938096e+00+2.90276874e+00j,
        -2.21872731e-01+1.37715718e+00j],
       [ 3.90382480e-01+2.19301101e-01j, -1.14505140e-01-2.59564098e+00j,
         3.24934449e-01-5.39172011e-01j],
       [-5.51586505e-01+2.00067692e-01j, -6.19589136e-01+1.57711540e+00j,
        -1.46160646e-01+2.99227053e-01j],
       [-2.20900153e-01-4.75193480e-01j,  4.48097454e-01-6.01293561e-01j,
        -4.50843056e-03-2.46228847e-01j],
       [ 5.69319605e-01-2.67559032e-01j,  2.50509287e-01+4.70366130e-01j,
         5.21296543e-02+3.66214535e-01j],
       [-8.14002444e-02+8.12741779e-01j,  6.22817100e-02-4.99131372e-01j,
         7.53080473e-02-4.48323133e-01j],
       [-6.14622831e-01-6.62524539e-01j, -1.43254704e-01-6.51335178e-02j

In [7]:
lags = [256, 128, 64, 32, 16, 8, 4, 2]
for lag in lags:
    extract_feature('../SMNI_CMI_TEST_CSV', lag)
    extract_feature('../SMNI_CMI_TRAIN_CSV', lag)

KeyboardInterrupt: 

In [15]:
data = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=range(3,259))
channel_name = np.loadtxt('../out.csv', delimiter=",", skiprows=1, usecols=1, dtype='str', encoding='utf-8')

df_data = pd.DataFrame(data.T, columns=channel_name)
df_data = df_data.drop(columns=['X', 'Y', 'nd'])
bispectrum = []
for channel in df_data.columns:
    magnitude = calcCumulantOrde3(df_data[channel], t, 256)
    bispectrum.append(magnitude)
bispectrum = np.array(bispectrum)
np.save('../bispectrum.npy', bispectrum)
x = np.load('../bispectrum.npy')

(bispectrum == x).all()

True